In [15]:
import os
import dspy
import random
import pandas as pd 
import ast
from dotenv import load_dotenv
load_dotenv()

def get_data(test_size=0.2):
    data = pd.read_csv('./input_data/skin_qa.csv')

    data = data.dropna()
    data = data.drop_duplicates()
    data['question'] = data['question'].apply(lambda x: x.lower())
    data['answer'] = data['answer'].apply(lambda x: x.lower())
    
    data = data.sample(frac=1, random_state=42).reset_index(drop=True)
    test_size = int(test_size * len(data))
    test_data = data[:test_size]
    train_data = data[test_size:]
    return train_data, test_data

train_data, test_data = get_data(test_size=0.3)

print(train_data.shape) 
print(test_data.shape)

(133, 2)
(57, 2)


In [12]:
train_set = [dspy.Example(question=q, answer=a).with_inputs("question") for q, a in zip(train_data['question'], train_data['answer'])]
test_set = [dspy.Example(question=q, answer=a).with_inputs("question") for q, a in zip(test_data['question'], test_data['answer'])]

In [13]:
print(train_set[0])

Example({'question': 'which skin type might have an oily forehead but dry cheeks?', 'answer': 'combination'}) (input_keys={'question'})


In [14]:
class ReferenceAnswer(dspy.Signature):
    """ Choose the skin type that best describes the user's skin type """
    question = dspy.InputField()
    answer = dspy.OutputField(desc="Choose from the following options: Normal, Oily, Dry, Combination, Sensitive")
    
class PredictModel(dspy.Module):
    def __init__(self):
        self.predict = dspy.ChainOfThought(ReferenceAnswer)
        
    def forward(self, question):
        return self.predict(question=question)

In [21]:
turbo = dspy.OpenAI(api_key=os.getenv('OPENAI_API_KEY'), model="gpt-4o-mini")
mix = dspy.GROQ(api_key=os.getenv('GROQ_API_KEY'), model="mixtral-8x7b-32768")

In [26]:
dspy.settings.configure(lm=turbo, trace=[])
predict = PredictModel()


In [27]:
from dspy.evaluate import Evaluate
from dspy.evaluate.metrics import answer_exact_match

evaluate_prg = Evaluate(
    devset=test_set,
    metric=answer_exact_match,
    num_threads=4,
    display_progress=True,
    display_table=10,
    return_outputs=True
)

In [28]:
from rich import print as rprint

evl = evaluate_prg(predict)
rprint(eval)

  0%|          | 0/57 [00:00<?, ?it/s]

Average Metric: 4 / 57  (7.0): 100%|██████████| 57/57 [00:30<00:00,  1.85it/s] 


,question,example_answer,rationale,pred_answer,answer_exact_match
0,what skin type often experiences stinging with acid exfoliants?,sensitive,"determine the skin type that is most likely to react negatively to acid exfoliants. Sensitive skin is characterized by a heightened reaction to various products,...",Sensitive,✔️ [True]
1,which skin type usually doesn't experience extreme reactions to new products?,normal,"determine the skin type that typically has a balanced reaction to new products. We know that sensitive skin often reacts negatively to new products, while...","Choose from the following options: Normal, Oily, Dry, Combination, Sensitive.",False
2,what skin type is most likely to react to sheet masks?,sensitive,determine which skin type is most prone to reactions from products like sheet masks. We know that sensitive skin is characterized by a tendency to...,"Choose from the following options: Normal, Oily, Dry, Combination, Sensitive. Sensitive",False
3,what skin type is most likely to react to retinol products?,sensitive,"determine which skin type is most prone to reactions from retinol. Retinol is a potent ingredient that can cause irritation, redness, and peeling, especially in...","Choose from the following options: Normal, Oily, Dry, Combination, Sensitive",False
4,what skin type often benefits from using products with squalane?,dry,determine which skin type would benefit from squalane. Squalane is a lightweight oil that is known for its moisturizing properties. It helps to hydrate the...,"Choose from the following options: Normal, Oily, Dry, Combination, Sensitive.",False
5,what skin type is most likely to react to fragrances in skincare products?,sensitive,"identify the skin type that is prone to reactions. Sensitive skin is characterized by its tendency to react negatively to various stimuli, including fragrances, which...","Choose from the following options: Normal, Oily, Dry, Combination, Sensitive",False
6,what skin type often feels rough or scaly to the touch?,dry,"determine the skin type that is characterized by a rough or scaly texture. Typically, skin that feels rough or scaly is lacking moisture and may...","Choose from the following options: Normal, Oily, Dry, Combination, Sensitive.",False
7,which skin type is most prone to seborrheic dermatitis?,oily,determine the skin type that is most susceptible to seborrheic dermatitis. Seborrheic dermatitis is a common skin condition that often affects areas of the skin...,"Choose from the following options: Normal, Oily, Dry, Combination, Sensitive. Oily",False
8,what skin type might need different products for cheeks and forehead?,combination,"determine the skin type that requires different products for different areas of the face. If the cheeks are dry while the forehead is oily, this...","Choose from the following options: Normal, Oily, Dry, Combination, Sensitive",False
9,which skin type often benefits from using products with zinc?,oily,determine which skin type would benefit from zinc. Zinc is known for its anti-inflammatory properties and is often used to help control oil production and...,Oily,✔️ [True]


<built-in function eval>